In [1]:
%matplotlib inline

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 7.0810e-01, -2.0234e-01, -2.7697e-02, -9.0180e-02,  6.0383e-01,
          3.8660e-01, -6.3321e-02, -6.4357e-01,  1.3363e-01, -8.8465e-01,
          7.4847e-02,  7.2338e-01, -1.7347e-01, -9.6468e-02,  2.2572e-01,
          7.0648e-01, -2.6426e-01,  2.4609e-01, -7.5594e-02,  2.4092e-01],
        [ 2.0843e-01, -4.3973e-02,  5.6758e-02,  1.1729e-01,  3.3175e-01,
         -5.4562e-02, -1.4041e-01, -5.2778e-01, -2.9531e-01, -6.5065e-01,
          3.7667e-01,  2.3575e-01, -8.9535e-02, -7.1310e-02, -1.1409e-01,
          3.8473e-01, -2.9001e-02,  7.3168e-02,  2.1666e-01, -2.8219e-01],
        [ 1.2027e-01, -5.4479e-02, -3.4571e-02,  8.6609e-02,  2.6646e-01,
          2.1801e-01,  1.3120e-05, -2.4486e-01, -4.0915e-02, -5.6223e-01,
          4.0318e-01,  1.1474e+00, -2.5827e-01, -1.5660e-01, -1.6509e-01,
          4.5950e-01, -3.2235e-01, -4.1041e-02, -1.1736e-01, -2.5513e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[7.0810e-01, 0.0000e+00, 0.0000e+00, 0.0

In [10]:
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20, 10))
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0021,  0.0258, -0.0276,  ...,  0.0315, -0.0089,  0.0307],
        [ 0.0139,  0.0249,  0.0272,  ..., -0.0157, -0.0191, -0.0050]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0071, -0.0012], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0160, -0.0256, -0.0424,  ..., -0.0171, -0.0241,  0.0239],
        [ 0.0402,  0.0244,  0.0249,  ...,  0.0192,  0.0411,  0.0119]],
       device='cuda:0', grad_fn=<Sl